In [63]:
import data, data_utils
import importlib as I
I.reload(data_utils)

<module 'data_utils' from '/home/suriya/_/tf/TF/simple-seq2seq/data_utils.py'>

In [3]:
data_ctl, idx_words, idx_phonemes = data.load_data()
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_words, idx_phonemes)

In [32]:
# parameters 
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 128
xvocab_size = len(data_ctl['idx2alpha'].keys())  # 27 - 1 = 26
yvocab_size = len(data_ctl['idx2pho'].keys())  # 70 - 1 = 69
emb_dim = 128

In [13]:
import tensorflow as tf
import numpy as np

In [33]:
tf.reset_default_graph()

In [34]:
enc_ip = [ tf.placeholder(dtype=tf.int32,
                       shape = (None,),
                       name = 'ei_{}'.format(i)) for i in range(xseq_len) ]
# alternatively
#  enc_ip = tf.placeholder(shape=[None,xseq_len], dtype=tf.int32, name='enc_ip')
labels = [ tf.placeholder(dtype=tf.int32,
                       shape = (None,),
                       name = 'ei_{}'.format(i)) for i in range(yseq_len) ]
# alternatively
#  labels = tf.placeholder(shape=[None,yseq_len], dtype=tf.int32, name='labels')
dec_ip = [ tf.zeros_like(enc_ip[0], dtype=tf.int32, name='GO')] + labels[:-1]

In [35]:
keep_prob = tf.placeholder(tf.float32)
basic_cell = tf.nn.rnn_cell.DropoutWrapper(
        tf.nn.rnn_cell.BasicLSTMCell(emb_dim, state_is_tuple=True),
        output_keep_prob=keep_prob)
stacked_lstm = tf.nn.rnn_cell.MultiRNNCell([basic_cell]*3, state_is_tuple=True)


with tf.variable_scope('decoder') as scope:
    decode_outputs, decode_states = tf.nn.seq2seq.embedding_rnn_seq2seq(enc_ip,dec_ip, stacked_lstm,
                                        xvocab_size, yvocab_size, emb_dim)
    scope.reuse_variables()
    # testing
    decode_outputs_test, decode_states_test = tf.nn.seq2seq.embedding_rnn_seq2seq(
        enc_ip, dec_ip, stacked_lstm, xvocab_size, yvocab_size,emb_dim,
        feed_previous=True)

In [36]:
# we weight the losses based on timestep of decoder output
loss_weights = [tf.ones_like(l, dtype=tf.float32) for l in labels] # gives [1, 1, ..., 1,1] - equal weights
loss = tf.nn.seq2seq.sequence_loss(decode_outputs, labels, loss_weights, yvocab_size)
train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss)

In [30]:
def get_feed(X, Y):
    feed_dict = {enc_ip[t]: X[t] for t in range(xseq_len)}
    feed_dict.update({labels[t]: Y[t] for t in range(yseq_len)})
    return feed_dict

In [37]:
# training session
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    # create a generator
    train_batch_gen = data_utils.batch_gen(trainX, trainY, batch_size)
    X, Y = train_batch_gen.__next__()
    feed_dict = get_feed(X, Y)
    feed_dict[keep_prob] = 0.5
    _, out = sess.run([train_op, loss], feed_dict)

In [47]:
rtest_batch_gen = data_utils.rand_batch_gen(testX, testY, batch_size)
for i in range(10):
    batchX, batchY = rtest_batch_gen.__next__()
    print(i,batchX[40], batchY[40])

0 [19  3 18 15 12 12 19  0  0  0  0  0  0  0  0  0] [55 42 54 48 43 68  0  0  0  0  0  0  0  0  0  0]
1 [ 7 18  2  1 22  9  3  1  0  0  0  0  0  0  0  0] [33 26 19  2 65 35 42  7  0  0  0  0  0  0  0  0]
2 [ 8 25  4 18  1 20  5  4  0  0  0  0  0  0  0  0] [34 17 21 54 31 57  7 21  0  0  0  0  0  0  0  0]
3 [16 18  9 14 20 15 21 20 19  0  0  0  0  0  0  0] [53 54 36 45 57 15 57 55  0  0  0  0  0  0  0  0]
4 [20 18  1 14 19 16 15 18 20  5 18 19  0  0  0  0] [57 54  4 45 55 53 11 54 57 26 68  0  0  0  0  0]
5 [19  3  8 21  3  8  1 18  4 20  0  0  0  0  0  0] [56  8 42  1 54 57  0  0  0  0  0  0  0  0  0  0]
6 [16 18  5 19  3 18  9 16 20  9 15 14 19  0  0  0] [53 54  7 55 42 54 36 53 56  7 45 68  0  0  0  0]
7 [19 20  9 16 21 12  1 20  9 15 14  0  0  0  0  0] [55 57 37 53 67  7 43 30 56  7 45  0  0  0  0  0]
8 [18  5 24 18 15  4 20  0  0  0  0  0  0  0  0  0] [54 24 42 55 54 47 21  0  0  0  0  0  0  0  0  0]
9 [ 9 13  1  7  9 14  5  4  0  0  0  0  0  0  0  0] [37 44  5 41  7 45 21  0  0  0

In [66]:
data_utils.decode_word(batchX[12], data_ctl['idx2alpha'])

'pensioners'

In [67]:
data_utils.decode_phonemes(batchY[12], data_ctl['idx2pho'])

'P EH1 N SH AH0 N ER0 Z'

## Training

In [69]:
def train_batch(train_batch_gen):
    # get batches
    batchX, batchY = train_batch_gen.__next__()
    # build feed
    feed_dict = get_feed(batchX, batchY)
    feed_dict[keep_prob] = 0.5
    _, loss_v = sess.run([train_op, loss], feed_dict)
    return loss_v

In [111]:
def eval_step(eval_batch_gen):
    # get batches
    batchX, batchY = eval_batch_gen.__next__()
    # build feed
    feed_dict = get_feed(batchX, batchY)
    feed_dict[keep_prob] = 1.
    loss_v, dec_op_v = sess.run([loss, decode_outputs_test], feed_dict)
    # dec_op_v is a list; also need to transpose 0,1 indices
    dec_op_v = np.array(dec_op_v).transpose([1,0,2])
    return loss_v, dec_op_v, batchX, batchY

In [113]:
def eval_batch(eval_batch_gen, num_batches):
    losses, predict_loss = [], []
    for i in range(num_batches):
        loss_v, dec_op_v, batchX, batchY = eval_step(eval_batch_gen)
        losses.append(loss_v)
        for j in range(len(dec_op_v)):
            real = batchX.T[j]
            predict = np.argmax(dec_op_v, axis=2)[j]
            predict_loss.append(all(real == predict))
    return np.mean(losses), np.mean(predict_loss)

In [107]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    batchX, batchY = train_batch_gen.__next__()
    feed_dict = get_feed(batchX, batchY)
    feed_dict[keep_prob] = 1.
    loss_v, dec_op_val = sess.run([loss, decode_outputs_test], feed_dict)

In [112]:
dec_op_val[0].shape

(128, 70)

In [98]:
#train_batch_gen = data_utils.rand_batch_gen(trainX, trainY, batch_size)
a,b = train_batch_gen.__next__()
print(a.shape,b.shape)

(128, 16) (128, 16)


In [119]:
import sys

In [ ]:
val_batch_gen = data_utils.rand_batch_gen(validX, validY, 16)
train_eval_batch_gen = data_utils.rand_batch_gen(trainX, trainY, 16)
train_batch_gen = data_utils.rand_batch_gen(trainX, trainY, 128)

sess = tf.Session()
sess.run(tf.initialize_all_variables())

for i in range(100000):
    try:
        train_batch(train_batch_gen)
        if i % 1000 == 0:
            val_loss, val_predict = eval_batch(val_batch_gen, 16)
            train_loss, train_predict = eval_batch(train_eval_batch_gen, 16)
            print("val loss   : {0}, val predict   = {1}%".format(val_loss, val_predict * 100))
            print("train loss : {0}, train predict = {1}%".format(train_loss, train_predict * 100))
            print
            sys.stdout.flush()
    except KeyboardInterrupt:
        print("interrupted by user")
        break

val loss   : 4.202735900878906, val predict   = 0.0%
train loss : 4.201011657714844, train predict = 0.0%
